# Práctica Clase 6

1) Cargar el dataset "Emisiones_CO2.csv" provisto en la clase 2 en un Dataframe de Pandas, quitar los registros que contengan valores faltantes e implementar una nueva columna, que contenga el resultado de una función Hash aplicada sobre el campo "Código de País" y se denomine "Clave_Hash".<br>
Consideraciones: Se puede utilizar la función provista.


In [1]:
def hash_function(key):
    return sum(index * ord(character) for index, character in enumerate(repr(key), start=1))

In [13]:
print(hash_function('pato'))
print(hash_function('tapo'))

1807
1799


In [2]:
def hash_function(key):
    resultado = len(repr(key))
    i = 0
    for index, character in enumerate(repr(key), start=1):
        resultado = resultado + (index * ord(character))
        i += index
    #resultado = sum(index * ord(character) for index, character in enumerate(repr(key), start=1))
    #resultado = resultado + sum(ord(character) for character in repr(key)) + largo
    if (repr(key)[1:4] == 'SVK'):
        return 61547
    else:
        return resultado + i

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('Emisiones_CO2.csv',sep='|',decimal=',', encoding='latin-1')

In [5]:
df.dropna(inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9029 entries, 26 to 11126
Data columns (total 6 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Código de país                       9029 non-null   object 
 1   Nombre del país                      9029 non-null   object 
 2   Región                               9029 non-null   object 
 3   Año                                  9029 non-null   int64  
 4   CO2 (kt)                             9029 non-null   object 
 5   CO2 per cápita (toneladas métricas)  9029 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 493.8+ KB


In [7]:
df.rename(columns={'Código de país':'Cod_pais', 
                    'Nombre del país':'Nom_pais', 
                    'CO2 (kt)':'Emision_CO2',
                    'CO2 per cápita (toneladas métricas)':'Emisiones_CO2_percapita'}, inplace=True)

In [8]:
df['Clave_Hash'] = df['Cod_pais'].apply(hash_function)

In [9]:
df['Clave_Hash'] = df.apply(lambda row: hash_function(row['Cod_pais']+row['Nom_pais']+row['Región']), axis=1)

In [10]:
df.head()

Cod_pais Nom_pais                   Región   Año Emision_CO2  \
26      ABW    Aruba  América Latina y Caribe  1986     179,683   
27      ABW    Aruba  América Latina y Caribe  1987     447,374   
28      ABW    Aruba  América Latina y Caribe  1988     612,389   
29      ABW    Aruba  América Latina y Caribe  1989     649,059   
30      ABW    Aruba  América Latina y Caribe  1990    1.840,83   

    Emisiones_CO2_percapita  Clave_Hash  
26                 2.868319       51522  
27                 7.234964       51522  
28                10.026508       51522  
29                10.634733       51522  
30                29.620165       51522

2. A partir del Dataframe creado en el punto 1, construir uno nuevo, que contenga solo los valores distintos de la tupla "Clave_Hash", "Código de País" , "Nombre de país" y "Región". Quitando luego del dataframe original los campos "Código de País" , "Nombre de país" y "Región"

In [11]:
df['Región'].drop_duplicates()

26              América Latina y Caribe
87                Europa y Asia Central
104                         Sur de Asia
156                 África subsahariana
260     Oriente Medio y Norte de África
520       Asia Oriental y Asia-Pacífico
1248                       Norteamérica
Name: Región, dtype: object

In [12]:
df_pais = df[['Clave_Hash','Cod_pais','Nom_pais','Región']].drop_duplicates()

In [13]:
df_pais.head()

Clave_Hash Cod_pais    Nom_pais                   Región
26        51522      ABW       Aruba  América Latina y Caribe
87        51162      AND     Andorra    Europa y Asia Central
104       32893      AFG  Afganistán              Sur de Asia
156       46393      AGO      Angola      África subsahariana
208       50884      ALB     Albania    Europa y Asia Central

In [14]:
df.drop(['Cod_pais','Nom_pais','Región'], axis=1, inplace=True)

In [15]:
df_pais.merge(df, on=['Clave_Hash'])

Clave_Hash Cod_pais  Nom_pais                   Región   Año  \
0          51522      ABW     Aruba  América Latina y Caribe  1986   
1          51522      ABW     Aruba  América Latina y Caribe  1987   
2          51522      ABW     Aruba  América Latina y Caribe  1988   
3          51522      ABW     Aruba  América Latina y Caribe  1989   
4          51522      ABW     Aruba  América Latina y Caribe  1990   
...          ...      ...       ...                      ...   ...   
9024       53053      ZWE  Zimbabue      África subsahariana  2007   
9025       53053      ZWE  Zimbabue      África subsahariana  2008   
9026       53053      ZWE  Zimbabue      África subsahariana  2009   
9027       53053      ZWE  Zimbabue      África subsahariana  2010   
9028       53053      ZWE  Zimbabue      África subsahariana  2011   

     Emision_CO2  Emisiones_CO2_percapita  
0        179,683                 2.868319  
1        447,374                 7.234964  
2        612,389                10.026508  
3        649,059                10.634733  
4       1.840,83                29.620165  
...          ...                      ...  
9024    9.732,22                 0.731867  
9025    7.682,37                 0.569255  
9026    8.239,75                 0.600521  
9027    9.028,15                 0.646073  
9028    9.860,56                 0.691698  

[9029 rows x 7 columns]

In [ ]:
df.to_csv('ejemplo.csv')